In [1]:
import pandas as pd
import glob
import os

In [2]:
''' Delete existing converted files '''

pattern = '../seed_datasets/**/*.csv' # Pattern to match all csv files

# Recursively match files in subdirectories
for filepath in glob.glob(pattern, recursive=True):
    if filepath.endswith('-converted.csv') or filepath.endswith('-title.txt'):
        os.remove(filepath)  # Delete the file

# Check for remaining files
remaining_converted_files = [f for f in glob.glob(pattern, recursive=True) if f.endswith('-converted.csv') or f.endswith('-title.txt')]
remaining_converted_files

[]

In [3]:
''' Convert data string to a DataFrame and a title '''
def string_to_dataframe_and_title(data_string):
    # Split the data into rows
    rows = data_string.strip().split("<0x0A>")
    
    title = None
    header = None

    # Check if the first row is a title and extract it
    if "TITLE |" in rows[0]:
        potential_title = rows[0].split(" | ", 1)[1]  # Split and take second part as the potential title
        if potential_title.strip().lower() not in ['', 'title']: # Handle cases with placeholder titles
            title = potential_title  # Only set title if its meaningful
        rows.pop(0)  # Remove the title row from processing

    # The next row should be the header
    if rows:
        header = rows.pop(0).split(" | ")
    else:
        raise ValueError("No header row found after the title row.")

    # Check if there are any data rows left after removing title and header
    if not rows:
        raise ValueError("No data rows found after the header row.")

    # Split remaining rows based on number of headers
    records = [row.strip().split(" | ") for row in rows]
    
    # Create df and convert data to numeric where possible
    df = pd.DataFrame(records, columns=header).apply(pd.to_numeric, errors='ignore')

    return df, title

In [4]:
''' Save the title to a separate txt file '''
def save_title_to_file(title, filepath):
    if title:
        title_filepath = f"{filepath.rsplit('.', 1)[0]}-title.txt" # Add '-title' extention
        with open(title_filepath, 'w') as f:
            f.write(title)

In [5]:
# Pattern for matching csvs in directory and its subdirectories
pattern = '../seed_datasets/**/*.csv'  

for filepath in glob.glob(pattern, recursive=True):
    with open(filepath, 'r') as file:
        data_string = file.read()

    try:
        df, title = string_to_dataframe_and_title(data_string)
        
        # Create new file name with '-converted' extention
        base, extension = os.path.splitext(filepath)
        new_filepath = f"{base}-converted{extension}"
        
        # Save the DataFrame
        df.to_csv(new_filepath, index=False)
            
        # Save the title to a separate file
        save_title_to_file(title, filepath)
    except ValueError as e:
        print(f"Error processing {filepath}: {e}")

In [6]:
''' Check if CSV was converted properly '''
def test_csv_conversion(directory_pattern):
    not_converted_properly = []

    # Loop through each file in directory
    for filepath in glob.glob(directory_pattern, recursive=True):
        # Only check converted files
        if filepath.endswith('-converted.csv'):
            try:
                df = pd.read_csv(filepath)
                # If shape is (0, 1), add to list
                if df.shape == (0, 1):
                    not_converted_properly.append(filepath)
            except Exception as e: # If error reading CSV, add to list
                not_converted_properly.append(filepath)

    return not_converted_properly

csvs_not_converted_properly = test_csv_conversion('../seed_datasets/**/*.csv')
csvs_not_converted_properly

[]